In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from bs4 import BeautifulSoup 
import urllib
import requests
import html5lib
import json
import traceback

import time
import numpy as np

import os

In [129]:
p = 1
url = f"https://xeno-canto.org/explore?dir=0&order=cnt&pg={p}"
req = requests.get(url)
soup = BeautifulSoup(req.text, "html.parser")
found = soup.findAll("tbody")
print(soup)


<!DOCTYPE html>

<html>
<head>
<title>Browse Recordings :: page 1 :: xeno-canto</title>
<meta charset="utf-8"/>
<meta content="index,follow" name="robots"/>
<meta content="bird songs, bird sounds, birds, bird sound recordings, bird song recordings, bird song archive, bird sound archive, creative commons, xeno-canto" name="keywords"/>
<link href="/static/img/favicon.png" rel="icon" type="image/png"/>
<link href="/static/img/favicon-60.png" rel="apple-touch-icon" type="image/png"/>
<link href="/static/img/favicon-76.png" rel="apple-touch-icon" sizes="76x76" type="image/png"/>
<link href="/static/img/favicon-120.png" rel="apple-touch-icon" sizes="120x120" type="image/png"/>
<link href="/static/img/favicon-152.png" rel="apple-touch-icon" sizes="152x152" type="image/png"/>
<link href="/static/js/jquery.fancybox.css?1.0.13" rel="stylesheet" type="text/css"/>
<link href="/static/js/jquery.qtip.min.css?1.0.13" rel="stylesheet" type="text/css"/>
<link href="/static/js/jplayer/xcplayer.css?1.0.

In [130]:
soup = BeautifulSoup(req.content, 'html5lib')
print(soup.prettify())

<!DOCTYPE html>
<html>
 <head>
  <title>
   Browse Recordings :: page 1 :: xeno-canto
  </title>
  <meta charset="utf-8"/>
  <meta content="index,follow" name="robots"/>
  <meta content="bird songs, bird sounds, birds, bird sound recordings, bird song recordings, bird song archive, bird sound archive, creative commons, xeno-canto" name="keywords"/>
  <link href="/static/img/favicon.png" rel="icon" type="image/png"/>
  <link href="/static/img/favicon-60.png" rel="apple-touch-icon" type="image/png"/>
  <link href="/static/img/favicon-76.png" rel="apple-touch-icon" sizes="76x76" type="image/png"/>
  <link href="/static/img/favicon-120.png" rel="apple-touch-icon" sizes="120x120" type="image/png"/>
  <link href="/static/img/favicon-152.png" rel="apple-touch-icon" sizes="152x152" type="image/png"/>
  <link href="/static/js/jquery.fancybox.css?1.0.13" rel="stylesheet" type="text/css"/>
  <link href="/static/js/jquery.qtip.min.css?1.0.13" rel="stylesheet" type="text/css"/>
  <link href="/stati

In [3]:
path_info = os.path.join('./data', 'info_wo_audio')
path_sound = os.path.join('./data', 'sound_db')
def save_data(d, counter, save_audio=True):
    if save_audio:
        with open(os.path.join(path_sound, f'sound_{counter}.mp3'), "wb") as f_out:
            f_out.write(requests.get(f"http:{d['data-xc-filepath']}").content)
    json_object = json.dumps(d, indent = 4)
    with open(os.path.join(path_info, f'info_{counter}.json'), "w") as outfile:
        outfile.write(json_object)

In [ ]:
counter = 0
for p in range(0, 22577):
    url = f"https://xeno-canto.org/explore?dir=0&order=cnt&pg={p}"
    req = requests.get(url)
    soup = BeautifulSoup(req.content, 'html5lib')

    # time.sleep(1)
    table = soup.findAll(lambda tag: tag.name=='tbody')
    if table and len(table)>1:
        rows = table[1].findAll(lambda tag: tag.name=='tr')
        for ri, _row in enumerate(rows):
            if _row:
                try:
                    # time.sleep(np.random.randint(700)/1000.)
                    name_item = _row.find(attrs={'class' : "common-name"})
                    name = name_item.find_next()
                    t = _row.find_all(attrs={"class": "jp-type-single"})
                    d = {
                        'name': name.text,
                        'sci_name': name.attrs['href'].replace('/species/',  ''),
                        'id': t[0].attrs['id'],
                        'data-xc-filepath':  t[0].attrs['data-xc-filepath'], 
                        'data-xc-id':  t[0].attrs['data-xc-id'],
                        'page': p,
                        'row': ri
                    }

                    for i, td in enumerate(_row.findAll("td")):
                        if i==2:
                            try:
                                d['length'] = td.text.strip()
                            except:
                                d['length'] = None
                        if i==3:
                            try:
                                d['contributor'] = td.find_next().text
                            except:
                                d['contributor'] = None
                        if i==4:
                            try:
                                d['date'] = td.text
                            except:
                                d['date'] = None 
                        if i==5:
                            try:
                                d['time'] = td.text
                            except:
                                d['time'] = None                                
                        if i==6:
                            try:
                                d['country'] = td.text
                            except:
                                d['country'] = None 
                        if i==7:
                            try:
                                d['location'] = td.find_next().text
                            except:
                                d['location'] = None
                        if i==8:
                            try:
                                d['elev'] = td.text
                            except:
                                d['elev'] = None 
                        if i==9:
                            try:
                                d['type'] = td.text
                            except:
                                d['type'] = None
                        if i==10:
                            try:
                                text = td.find_next().text.split('\n')
                                d['note'] = text[0]
                                for t in text:
                                    if 'bird-seen:' in t:
                                        d['bird_seen'] = t.split(':')[1]
                                    if 'playback-used:' in t:
                                        d['playback_used'] = t.split(':')[1]
                            except:
                                d['note'] = None 
                        if i==11:
                            try:
                                d['class'] = td.find(attrs={'class':'selected'}).find_next().text
                            except:
                                d['class'] = None 
                    if d['class'] == 'A':
                        save_data(d=d, counter=counter, save_audio=False)
                        counter += 1
                except Exception as e:
                    print(p, _row, traceback.format_exc())
